In [69]:


# 리퀘스트 라이브러리로 html 소스받아오기

# 뷰티풀수프로 원하는 데이터 파싱

# 판다스 활용, csv로 저장하기 
import requests
from bs4 import BeautifulSoup
import pandas as pd

page = 1

while(True) :
    
    rq_data = {
        'currentPage': page,
        'maxResults':'', 
        'maxLinks': '',
        'sort': '',
        'series':'' ,
        'pageGrouping': 'Y',
        'mdayCnt': '0',
        'selectDate': '2020.06.29',
        'textCrpCik':'',
    }
    url_post = "http://dart.fss.or.kr/dsac001/search.ax"
    rq = requests.post(url_post, data=rq_data)

    html = rq.text

    # print(html)

    soup = BeautifulSoup(html, 'html.parser')

    try : 
        table = soup.select("div.table_list > table")[0]

        trs = table.select("tr")
        print(len(trs))


        list_rows = []

        # 테이블 항목 선택
        for tr in trs[1:] : 
            time = tr.select("td")[0].text
            company = tr.select("td")[1].text
            paper_name = tr.select("td")[2].text
            paper_url = tr.select("td:nth-child(3) > a")[0]['href']
            submit = tr.select("td")[3].text
            submit_date = tr.select("td")[4].text
            try : 
                etc = tr.select("td:nth-child(6) > img")[0]['title']
            except :
                etc = '-'

            tr_dict = {
                "time" : time,
                "company" : company,
                "paper_name" : paper_name,
                "paper_url" : "http://dart.fss.or.kr" +  paper_url,
                "submit" : submit,
                "submit_date" : submit_date,
                "etc" : etc,
            }

            for key_iter in tr_dict.keys() :
                tr_dict[key_iter] = tr_dict[key_iter].replace('\t','')   
                tr_dict[key_iter] = tr_dict[key_iter].replace('\n','')   
                tr_dict[key_iter] = tr_dict[key_iter].replace('\r','')   
                tr_dict[key_iter] = tr_dict[key_iter].replace('\xa0','')   

            list_rows.append(tr_dict)

        df = pd.DataFrame(list_rows)
        df.to_csv(str(page) + "dart.csv", encoding='utf-8-sig')
        page = page + 1
    except :
        break
    
    
    

101
31
2
